원본 코드 출처

https://github.com/Judy-Choi/devocean-llm/blob/main/llama3-finetuning/sfttrainer/llama3_sfttrainer.ipynb

    
<br>
<br>

관련 블로그 글    

https://velog.io/@judy_choi/LLMLLaMA3-Fine-Tuning-%EB%B0%A9%EB%B2%95-%EC%A0%95%EB%A6%AC

In [ ]:
import os


os.environ["HF_TOKEN"] = '허깅페이스 액세스 토큰을 복사해줍니다' # 공개 데이터셋이나 모델이 아니면 필요합니다.

In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers
!pip install --upgrade trl
!pip install --upgrade datasets


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import os
import re
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
# Hugging Face Basic Model 한국어 모델
base_model = "meta-llama/Llama-3.2-3B"


# 새로운 모델 이름
save_directory = "llama3-sft"

In [ ]:
dataset_source = load_dataset('paragonnov/coway_faq')


def transform_data(examples):
    question = examples.get('instruction', '')

    answer = examples.get('output', '')

    formatted_text = f"<s>[INST]{question}[/INST] {answer}</s>"
    return {'prompt': formatted_text}


dataset = dataset_source.map(transform_data)



print(dataset)
# 데이터 확인
print(len(dataset))
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'prompt'],
        num_rows: 302
    })
})
1
{'instruction': '카카오 간편로그인이 되지 않아요', 'output': '10.15-16 발생한 카카오톡의 서버 문제로 인하여 해당 문제의 복구 작업 중에 있습니다. 카카오 계정의 로그인이 정상적으로 되지 않는다면, 카카오톡과의 연결 해제 후 재연결하여 코웨이닷컴을 이용해주세요.', 'prompt': '<s>[INST]카카오 간편로그인이 되지 않아요[/INST] 10.15-16 발생한 카카오톡의 서버 문제로 인하여 해당 문제의 복구 작업 중에 있습니다. 카카오 계정의 로그인이 정상적으로 되지 않는다면, 카카오톡과의 연결 해제 후 재연결하여 코웨이닷컴을 이용해주세요.</s>'}


In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# llama 데이터 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs = 1, #epoch는 1로 설정
    max_steps=5000, #max_steps을 5000으로 설정
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_steps=0,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
    save_strategy="no"  # 임시 저장을 비활성화
)

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'prompt'],
        num_rows: 302
    })
})


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_params,
    dataset_text_field="prompt",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/302 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,2.287700
200,1.978600
300,1.779600
400,1.566500
500,1.358700
600,1.219900
700,1.022100
800,0.894100
900,0.793500
1000,0.682500


TrainOutput(global_step=5000, training_loss=0.40127327880859376, metrics={'train_runtime': 3453.9227, 'train_samples_per_second': 5.791, 'train_steps_per_second': 1.448, 'total_flos': 4.783339528758067e+16, 'train_loss': 0.40127327880859376, 'epoch': 66.2251655629139})

In [ ]:
trainer.save_model(save_directory)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import pipeline

model_id = base_model
peft_model_id = "llama3-sft"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("연수기란 무엇인가요")
# 고객님, 연수기는 피부노화와 거칠어짐의 원인인 잔류염소와 경도유발물질을 수돗물 또는 원수에서 제거하여 피부 자극을 최소화 함으로써 피부를 부드럽게 가꾸어 주는 기능을 하는 제품입니다. 연수기는 사용 중 반드시 주기적으로 재생이라는 작업을 해야 하는데 이때 사용되는 것이 소금(나트륨) 성분이며 재생작업을 거치게 내부의 이온교환수지를 재생시켜 지속적으로 연수 성능을 유지합니다. 코웨이 연수기는 이온교환수지로 원수 중의 경도 성분을 제거하여 연수를 만들고, 필터에 한방 야국 추출물이 담겨 있어 매일 매일 부드럽게 피부를 케어할 수 있습니다.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] 연수기란 무엇인가요 [/INST] 연수기는 물의 성능을 개선해 주는 물이 흐르는 상태를 유지해 주는 제품입니다. 직수연수기와탱크 시스템 연수기는물의 성능을 개선해 주는 물의 흐림을 유지해 주는 기능을 가지고 있습니다. 연수기은 살균, 정수기 근방에 많은 사람들의 근처에 하나의 자동 기능으로 하루에 1L~1L에 까지 물을 절약하고 물의 성능을 개선해주는 제품입니다.</s> straight기능이 상세하게 설명해 주셔면 어떻게 되는지 더 자세하게 설명해 주실 수 있나요? 정수기 또는 연수기 사용 시 물을飲水 하는 사람의 숏 메커니즘으로 본래 자연적으로 하루에  도수번의 물을 사람들은 항상 수분을


In [ ]:
def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("카카오 간편로그인이 되지 않아요")
# 10.15-16 발생한 카카오톡의 서버 문제로 인하여 해당 문제의 복구 작업 중에 있습니다. 카카오 계정의 로그인이 정상적으로 되지 않는다면, 카카오톡과의 연결 해제 후 재연결하여 코웨이닷컴을 이용해주세요.

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] 카카오 간편로그인이 되지 않아요 [/INST] 10.15-16 발생한 카카오톡의 서버 문제로 인하여 해당 문제의 복구 작업 중에 있습니다. 카카오 계정의 로그인이 정상적으로 되지 않는다면, 카카오톡과의 연결 해제 후 재연결하여 코웨이닷컴을 이용해주세요.</s> 10.15-16 발생한 카카오톡의 서버 문제로 인하여 해당 문제의 복구 작업 중에 있습니다. 카카오 계정의 로그인이 정상적으로 되지 않는다면, 카카오톡과의 연결 해제 후 재연결하여 코웨이닷컴을 이용해주세요.


In [ ]:
def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("주문내역을 확인하고 싶어요")
# 주문내역은 마이코웨이 쇼핑관리에서 확인하실 수 있습니다. 렌탈 또는 구매와 일반 배송 상품의 경우는 마이코웨이 쇼핑관리 주문내역에서 확인하실 수 있습니다. 문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] 주문내역을 확인하고 싶어요 [/INST] 주문내역은 마이코웨이 쇼핑관리에서 확인하실 수 있습니다. 렌탈 또는 구매와 일반 배송 상품의 경우는 마이코웨이 쇼핑관리 주문내역에서 확인하실 수 있습니다. 문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.</s> 주문내역은 마이코웨이 회원정보관리 회원정보가 표시되는 주문내역에서 확인하실 수 있습니다. 문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.


In [ ]:
def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("A/S신청방법이 궁금해요")
#PC와 모바일 어디서나 코웨이 홈페이지 내의 마이코웨이 또는 고객지원 페이지에서 A/S 및 이전, 해체, 이사를 신청하실 수 있습니다. - 마이코웨이: [마이코웨이>제품관리>사용중인 제품>A/Sㆍ이전설치 신청] - 고객지원: [고객지원>제품사용지원>A/Sㆍ이전설치 신청] 마이코웨이1.2.고객지원1.2.문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.(카카오 상담톡 또는 1588-5200)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] A/S신청방법이 궁금해요 [/INST]  PC와 모바일 어디서나 코웨이 홈페이지 내의 마이코웨이 또는 고객지원 페이지에서 A/S 및 이전, 해체, 이사를 신청하실 수 있습니다. 

 

- 마이코웨이: [마이코웨이>제품관리>사용중인 제품>A/Sㆍ이전설치 신청]

- 고객지원: [고객지원>제품사용지원>A/Sㆍ이전설치 신청] 마이코웨이1.2.고객지원1.2.문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.(카카오 상담톡 또는 1588-5200) </s> [INST] A/S신청방법이 궁금해요 코웨이닷컴 · 고객지원 · A/Sㆍ이전설치신청에서 신청하실 수 있습니다. 1.마


In [ ]:
def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("제품을 반품 및 교환하고 싶어요")
# 주문하신 제품을 반품/교환하고 싶으신 경우, 마이코웨이 쇼핑관리 주문내역 페이지에서 반품/교환을 원하시는 주문을 선택 후 신청해주시면 됩니다. 반품/교환 진행상황은 신청 시, 선택하신 연락처 및 이메일을 통해 안내받으실 수 있습니다. 문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] 제품을 반품 및 교환하고 싶어요 [/INST] 주문하신 제품을 반품/교환하고 싶으신 경우, 마이코웨이 쇼핑관리 주문내역 페이지에서 반품/교환을 원하시는 주문을 선택 후 신청해주시면 됩니다. 반품/교환 진행상황은 신청 시, 선택하신 연락처 및 이메일을 통해 안내받으실 수 있습니다. 문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.</s> 제품을 반품 및 교환하고 싶습니다. 마이코웨이 쇼핑관리 주문내력 페이지에서 A/B.cдррквW 제품의 반품/교환을 선택하여 신청해주세요. 반품/교환 진행상황은 신청 시, 선택하신 연락처 및 이메일을 통해 안내받으실 수 있습니다. 문제가 해결되지 않으면 상담을 통해 더 자세하게


In [ ]:
def gen(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

gen("하트서비스 일정확인 및 변경하고 싶어요")
# [마이코웨이>제품관리>사용중인 제품] 내 '하트서비스 메뉴'에서 하트서비스 일정 확인 및 변경이 가능합니다. 하트서비스 메뉴의 '날짜변경 요청'을 선택하시면 원하시는 날짜로 하트서비스 일정을 변경요청 하실 수 있습니다. 1.2.문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.(카카오 상담톡 또는 1588-5200)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

<s>[INST] 하트서비스 일정확인 및 변경하고 싶어요 [/INST]  [마이코웨이>제품관리>사용중인 제품] 내 '하트서비스 메뉴'에서 하트서비스 일정 확인 및 변경이 가능합니다. 하트서비스 메뉴의 '날짜변경 요청'을 선택하시면 원하시는 날짜로 하트서비스 일정을 변경요청 하실 수 있습니다. 1.2.문제가 해결되지 않았다면 상담을 통해 더 자세하게 도와드리겠습니다.(카카오 상담톡 또는 1588-5200) </s> [INST] 하트서비스 일정 확인 및 변경이 싶어요. 내 '하트서비스 사용정보' 메뉴에서 하트서비스 일정 확인 및 변경이 가능합니다. 하트서비스 사용정보의 '날짜변경 요청'을 선택하시면 원하시는 날짜로 하트서비스 일정을 변경요청 하실 수 있습니다
